# Python Power Measurement Tutorial

This tutorial shows how to perform a power measurement on the device.

The Hailo device supports power measurement which is done via the control protocol, if an INA231 is assembled on the board.

**Requirements:**

* Run the notebook inside the Python virtual environment: ```source hailo_virtualenv/bin/activate```
* Matplotlib for data visualization: ```pip install matplotlib```

**Attention:**

* These examples should run in a different process than the one that performs the actual inference.

When inside the ```virtualenv```, run ``jupyter-notebook <tutorial-dir>`` to open a Jupyter server that contains the tutorials (default folder on GitHub: ``hailort/libhailort/bindings/python/platform/hailo_tutorials/notebooks/``).

## Single Power Measurement

In [ ]:
%matplotlib inline
import time

from hailo_platform import Device, DvmTypes, PowerMeasurementTypes, SamplingPeriod, AveragingFactor, MeasurementBufferIndex # noqa F401


Initialize the hardware object:

In [ ]:
target = Device()

When using the ``power_measurement()`` function with no parameters, the function tries to detect which Hailo-10 board is connected (evaluation board, M.2 or mPCIe) and determine the DVM accordingly (at the moment only the boards referred to in this section have INA231 assembled and are supported). 

The parameter ``dvm`` (of type ``DvmTypes``) defines which DVM will be measured. The user can choose a specific DVM or choose the default DVM. The meaning of the default DVM changes according to the board or module in use. 

The default for the evaluation board is the sum of three DVMs: ``DvmTypes.VDD_CORE``, ``DvmTypes.MIPI_AVDD`` and ``DvmTypes.AVDD_H``. The sum of these three DVMs approximates the total power consumption of the chip in PCIe setups. Only power can be measured using this default option, as voltage and current cannot be totaled (or calculated) this way.

The default for platforms supporting current monitoring, such as M.2 and mPCIe modules, is ``DvmTypes.OVERCURRENT_PROTECTION``, which measures the power consumption of the whole module. 

See the API documentation for further details about the supported DVMs and measurement types.


In [ ]:
single_power_measurement = target.control.power_measurement()
print('Power from single measurement: {} W'.format(single_power_measurement))

## Periodic Power Measurement

In the following example, a periodic power measurement is taken.

A measurement index is selected. It is a member of the enum class MeasurementBufferIndex (between 0 and 3). The DVM is not given so the default DVM will be used, as explained above.


In [ ]:
buffer_index = MeasurementBufferIndex.MEASUREMENT_BUFFER_INDEX_0
target.control.set_power_measurement(buffer_index=buffer_index)

The following call to ``start_power_measurement()`` allows you to configure the power sampling frequency. In this case we keep the default configuration. The API documentation provides more details.

In [ ]:
target.control.start_power_measurement()

Clear the old samples and statistics (min, max, average) each time the measurement is taken from the chip.

In [ ]:
should_clear = True

The power measurement is read every second, for 10 seconds.

The firmware calculates the min, max, and average of all the values from the sensor. Note that the average calculated by the firmware is the "average of averages". This is the average of all values that have already been averaged by the sensor. The host then requests these values from the firmware every second by calling the ``get_power_measurement()`` function. The host can also read the average time interval between new sensor values.

In [ ]:
for _ in range(10):
    time.sleep(1)
    # Get saved power measurement values from the firmware.
    measurements = target.control.get_power_measurement(buffer_index=buffer_index, should_clear=should_clear)
    print('Average power is {} W. Min power is {} W. Max power is {} W.\nAverage time between power samples is {} mS\n'.format(measurements.average_value, measurements.min_value, measurements.max_value, measurements.average_time_value_milliseconds))

# Stop performing periodic power measurement
target.control.stop_power_measurement()

target.release()